In [1]:
import pandas as pd
import sqlite3
import pandas as pd


# Connect to the SQLite database
db_path = '../data/WARP.db'
conn = sqlite3.connect(db_path)

# Read the raw_NED_preds table into a DataFrame
df_NED_preds_processed = pd.read_sql_query("SELECT * FROM raw_NED_preds", conn)

# Close the database connection
conn.close()


import pandas as pd

# Zorg dat je datumkolommen correct zijn
df_NED_preds_processed['validfrom'] = pd.to_datetime(df_NED_preds_processed['validfrom'])
df_NED_preds_processed['validto'] = pd.to_datetime(df_NED_preds_processed['validto'])
df_NED_preds_processed = df_NED_preds_processed.set_index('lastupdate')
pd.to_datetime(df_NED_preds_processed.index)
df_NED_preds_processed.index = pd.to_datetime(df_NED_preds_processed.index)
df_NED_preds_processed.index



# Calculate rows where validfrom is earlier than lastupdate
mask = df_NED_preds_processed['validfrom'] >= df_NED_preds_processed.index
# Keep only rows that match this condition
print(f"Number of rows where validfrom is earlier than lastupdate: {(~mask).sum()}")



# Verwijder irrelevante kolommen
df_NED_preds_processed = df_NED_preds_processed.drop(columns=[
    '@id', '@type', 'id', 'point', 'granularity', 'granularitytimezone', 'activity', 'classification', 'capacity','percentage','emission','emissionfactor','validfrom',])
# remove rows with validto dates earlier than 2025-04-09 01:00:00+00:00
df_NED_preds_processed = df_NED_preds_processed[df_NED_preds_processed['validto'] > '2025-04-09 01:00:00+00:00']
# Rename current_datetime to 'fetch_timestamp'
df_NED_preds_processed = df_NED_preds_processed.rename(columns={'current_datetime': 'fetch_timestamp'})
#
df_NED_preds_processed['type'] = df_NED_preds_processed['type'].str.replace('/v1/types/', 'PRED_Type_')

print(df_NED_preds_processed.head())

# Save the processed DataFrame to a new SQLite database
#output_db_path = '../data/WARP.db'
#conn = sqlite3.connect(output_db_path)
#df_NED_preds_processed.to_sql('processed_NED_preds', conn, if_exists='replace', index=True)
#conn.close()

Number of rows where validfrom is earlier than lastupdate: 8229
                                  type  volume                   validto  \
lastupdate                                                                 
2025-04-04 22:45:27+00:00  PRED_Type_1  172140 2025-04-09 02:00:00+00:00   
2025-04-04 22:45:27+00:00  PRED_Type_1  150140 2025-04-09 03:00:00+00:00   
2025-04-04 22:45:27+00:00  PRED_Type_1  138338 2025-04-09 04:00:00+00:00   
2025-04-04 22:45:27+00:00  PRED_Type_1  182971 2025-04-09 05:00:00+00:00   
2025-04-04 22:45:27+00:00  PRED_Type_1  254562 2025-04-09 06:00:00+00:00   

                               fetch_timestamp  
lastupdate                                      
2025-04-04 22:45:27+00:00  2025-04-04 23:20:09  
2025-04-04 22:45:27+00:00  2025-04-04 23:20:09  
2025-04-04 22:45:27+00:00  2025-04-04 23:20:09  
2025-04-04 22:45:27+00:00  2025-04-04 23:20:09  
2025-04-04 22:45:27+00:00  2025-04-04 23:20:09  


In [2]:
# transforming forecasted electricity production data (e.g. from NED.nl) from long format to wide format, with explicit forecast horizon labels like pred_volume_type1_d1, pred_volume_type1_d2, etc.
df_NED_preds_processed["day_ahead"] = (df_NED_preds_processed["validto"].dt.normalize() - pd.to_datetime(df_NED_preds_processed.index).normalize()).dt.days
df_NED_preds_processed["pred_column"] = "volume_" + df_NED_preds_processed["type"].str.lower() + "_d" + df_NED_preds_processed["day_ahead"].astype(str)
df_NED_preds_processed["day_ahead"] = (df_NED_preds_processed["validto"].dt.normalize() - pd.to_datetime(df_NED_preds_processed.index).normalize()).dt.days
df_NED_preds_processed["pred_column"] = "volume_" + df_NED_preds_processed["type"].str.lower() + "_d" + df_NED_preds_processed["day_ahead"].astype(str)
# Group by validto and pred_column, taking the last value in case of duplicates
df_wide = df_NED_preds_processed.pivot_table(
    index="validto",
    columns="pred_column",
    values="volume",
    aggfunc=lambda x: x.dropna().iloc[-1] if not x.dropna().empty else np.nan #$x.dropna() removes NaNs, .iloc[-1] picks the last non-NaN value, if all values are NaN, it returns np.nan:
)

# Sort df_wide by validto index and display head and tail
print(df_wide.sort_index().head(5))
print("\nTail of df_wide (first 5 rows):")



pred_column                volume_pred_type_17_d0  volume_pred_type_17_d1  \
validto                                                                     
2025-04-09 02:00:00+00:00               1264364.0               1078600.0   
2025-04-09 03:00:00+00:00               1181091.0                953443.0   
2025-04-09 04:00:00+00:00                899096.0                844335.0   
2025-04-09 05:00:00+00:00               1004562.0                807438.0   
2025-04-09 06:00:00+00:00               1551765.0                949137.0   

pred_column                volume_pred_type_17_d2  volume_pred_type_17_d3  \
validto                                                                     
2025-04-09 02:00:00+00:00               1105664.0                977644.0   
2025-04-09 03:00:00+00:00               1024192.0                862923.0   
2025-04-09 04:00:00+00:00               1000249.0                825005.0   
2025-04-09 05:00:00+00:00               1151357.0                973646.0  

In [3]:
# dealing with NaN values, specifically those cells with numberical values in (both) adjacent cells
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# First make a copy to avoid modifying the original dataframe
df_filled = df_wide.copy()

# Ensure index is datetime for time-based interpolation
df_filled.index = pd.to_datetime(df_filled.index)

# For each column in the dataframe
for column in df_filled.columns:
    # Forward and backward fill to get values for interpolation
    series = df_filled[column]
    
    # Get the index as datetime for hourly interpolation
    index_as_datetime = pd.to_datetime(series.index)
    
    # Only interpolate if there's a value both before and after (linear interpolation)
    df_filled[column] = series.interpolate(
        method='time',
        limit=1, # limit=1 ensures we only interpolate for consecutive hours
        limit_direction='both' # limit_direction='both' ensures we only interpolate between actual values
    )

# Print some statistics about the filling
for column in df_filled.columns:
    filled_count = df_filled[column].count() - df_wide[column].count()
    if filled_count > 0:
        print(f"Filled {filled_count} NaN values in {column}")

Filled 27 NaN values in volume_pred_type_17_d0
Filled 1 NaN values in volume_pred_type_17_d1
Filled 1 NaN values in volume_pred_type_17_d2
Filled 1 NaN values in volume_pred_type_17_d3
Filled 1 NaN values in volume_pred_type_17_d4
Filled 1 NaN values in volume_pred_type_17_d5
Filled 1 NaN values in volume_pred_type_17_d6
Filled 38 NaN values in volume_pred_type_17_d7
Filled 27 NaN values in volume_pred_type_1_d0
Filled 1 NaN values in volume_pred_type_1_d1
Filled 1 NaN values in volume_pred_type_1_d2
Filled 1 NaN values in volume_pred_type_1_d3
Filled 1 NaN values in volume_pred_type_1_d4
Filled 1 NaN values in volume_pred_type_1_d5
Filled 1 NaN values in volume_pred_type_1_d6
Filled 38 NaN values in volume_pred_type_1_d7
Filled 27 NaN values in volume_pred_type_20_d0
Filled 1 NaN values in volume_pred_type_20_d1
Filled 1 NaN values in volume_pred_type_20_d2
Filled 1 NaN values in volume_pred_type_20_d3
Filled 1 NaN values in volume_pred_type_20_d4
Filled 1 NaN values in volume_pred_ty

In [4]:
import sqlite3  

# Save the filled DataFrame to a new SQLite database
output_db_path = '../data/WARP.db'
conn = sqlite3.connect(output_db_path)  
df_filled.to_sql('processed_NED_preds', conn, if_exists='replace', index=True)
conn.close()
# Check the filled DataFrame
print(df_filled.head())

pred_column                volume_pred_type_17_d0  volume_pred_type_17_d1  \
validto                                                                     
2025-04-09 02:00:00+00:00               1264364.0               1078600.0   
2025-04-09 03:00:00+00:00               1181091.0                953443.0   
2025-04-09 04:00:00+00:00                899096.0                844335.0   
2025-04-09 05:00:00+00:00               1004562.0                807438.0   
2025-04-09 06:00:00+00:00               1551765.0                949137.0   

pred_column                volume_pred_type_17_d2  volume_pred_type_17_d3  \
validto                                                                     
2025-04-09 02:00:00+00:00               1105664.0                977644.0   
2025-04-09 03:00:00+00:00               1024192.0                862923.0   
2025-04-09 04:00:00+00:00               1000249.0                825005.0   
2025-04-09 05:00:00+00:00               1151357.0                973646.0  

In [6]:
# Connect to the SQLite database and read the processed_NED_preds table, to verify the data
conn = sqlite3.connect('../data/WARP.db')
df = pd.read_sql_query("SELECT * FROM processed_NED_preds", conn)
conn.close()


print("DataFrame shape:", df.shape)
print("First five rows of the DataFrame:")
print(df.head())

DataFrame shape: (801, 57)
First five rows of the DataFrame:
                     validto  volume_pred_type_17_d0  volume_pred_type_17_d1  \
0  2025-04-09 02:00:00+00:00               1264364.0               1078600.0   
1  2025-04-09 03:00:00+00:00               1181091.0                953443.0   
2  2025-04-09 04:00:00+00:00                899096.0                844335.0   
3  2025-04-09 05:00:00+00:00               1004562.0                807438.0   
4  2025-04-09 06:00:00+00:00               1551765.0                949137.0   

   volume_pred_type_17_d2  volume_pred_type_17_d3  volume_pred_type_17_d4  \
0               1105664.0                977644.0               1294379.0   
1               1024192.0                862923.0               1302783.0   
2               1000249.0                825005.0               1350416.0   
3               1151357.0                973646.0               1535660.0   
4               1372373.0               1199476.0               1737786.0